In [1]:
import sys
import os

IN_COLAB = False
DATA_PATH = "data/"

# if in colab, make colab setup
if 'google.colab' in sys.modules:
    
    IN_COLAB = True
    SSH_DIR = '/root/.ssh'
    ZIP_PATH = "/content/drive/MyDrive/embedded_ml_data/VOCdevkit.zip"
    FILENAME = "dev-notebook.ipynb"
    BRANCH = "michael"

    # Setup ssh-auth to github
    try:
        os.makedirs(SSH_DIR)
    except FileExistsError:
        # directory already exists
        pass

    !ssh-keyscan github.com >> /root/.ssh/known_hosts
    !echo 'PUBKEY' > /root/.ssh/id_rsa.pub
    !echo -e "PRIVKEY" > /root/.ssh/id_rsa
    !chmod 644 /root/.ssh/known_hosts
    !chmod 600 /root/.ssh/id_rsa
    !ssh -T git@github.com

    # Setup working environment
    !rm -rf models
    !rm -rf utils
    !git clone -b $BRANCH https://github.com/yannickfunk/EmbeddedMLLab tmp
    !rm tmp/$FILENAME
    !mv tmp/* .
    !rm -rf tmp
    !rm -rf sample_data
    %pip install -r requirements.txt

    # Setup data
    from google.colab import drive
    drive.mount('/content/drive')
    try:
      os.makedirs('./data')
    except FileExistsError:
      # directory already exists
      pass
    !cp  $ZIP_PATH data/
    %pushd data
    !unzip -qq VOCdevkit.zip
    %popd
    drive.flush_and_unmount()

    
try:
    os.makedirs('./checkpoints')
    os.makedirs('./checkpoints/results')
except FileExistsError:
    # directory already exists
    pass


Prepare Tensorboard logger

In [4]:
# Setup logger
import pytorch_lightning as pl

LOGDIR='logs'
TENSORBOARD_DIR=LOGDIR+'/lightning_logs'

tensorboard = pl.loggers.TensorBoardLogger(save_dir=LOGDIR, default_hp_metric=True, log_graph=True)

try:
    os.makedirs('./'+LOGDIR+'/lightning_logs')
except FileExistsError:
    # directory already exists
    pass
 

In [5]:
%load_ext tensorboard
%tensorboard --logdir $TENSORBOARD_DIR

Reusing TensorBoard on port 6006 (pid 77464), started 18:13:43 ago. (Use '!kill 77464' to kill it.)

Prepare model

In [6]:
from models.tinyyolov2 import TinyYoloV2PersonOnly
import pytorch_lightning as pl
import torch

from utils.dataloader import VOCDataModule

from pytorch_lightning.callbacks.lr_monitor import LearningRateMonitor
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint

import nni
from nni.compression.pytorch import LightningEvaluator


# Setting up callbacks
lr_monitor = LearningRateMonitor(logging_interval='step')
early_stopping = EarlyStopping(monitor="val_loss", mode='min', verbose=True)
checkpointing = ModelCheckpoint(
    save_top_k=3,
    monitor="val_loss",
    mode="min",
    verbose=True,
    auto_insert_metric_name=True,
    save_last=True
)

trainer = nni.trace(pl.Trainer)(
    max_epochs=25,
    auto_lr_find=True,
    auto_scale_batch_size='binsearch',
    accelerator="auto",
    devices=[0] if torch.cuda.is_available() else None,
    accumulate_grad_batches=1,
    logger=tensorboard,
    log_every_n_steps=1,
    fast_dev_run= True if not torch.cuda.is_available() else False,
    callbacks=[
    lr_monitor,
    early_stopping,
    checkpointing
    ]
)


model = TinyYoloV2PersonOnly()
model.load_pt_from_disk(DATA_PATH + "/voc_pretrained.pt")

data = nni.trace(VOCDataModule)(person_only=True)

evaluator = LightningEvaluator(trainer, data)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Running in `fast_dev_run` mode: will run the requested loop using 1 batch(es). Logging and checkpointing is suppressed.


Tune and train model

In [ ]:
#trainer.tune(model, data)
trainer.fit(model, data)

In [ ]:
from utils.ap import precision_recall_levels, ap, display_roc
from utils.yolo import nms, filter_boxes
from utils.dataloader import VOCDataLoaderPerson

import torch
import numpy as np
import tqdm


test_loader = VOCDataLoaderPerson(train=False, batch_size=1, data_path=DATA_PATH, n_limit=350)

test_precision = []
test_recall = []

for inputs, targets in tqdm.tqdm(test_loader, total=350):
    with torch.no_grad():
        outputs = model(inputs)

    #The right threshold values can be adjusted for the target application
    outputs = filter_boxes(outputs, 0.0)
    outputs = nms(outputs, 0.5)
    outputs = torch.tensor(np.array(outputs))

    precision, recall = precision_recall_levels(targets[0], outputs[0])
    test_precision.append(precision)
    test_recall.append(recall)

avg_precision = ap(test_precision, test_recall)
print("average precision: ", avg_precision)
display_roc(test_precision, test_recall)